# 01_organizedata

这是MRI数据pipeline的第一部分: 整理数据。注意下面命令前面加！号是为了在python环境中跑bash命令

1. 上传数据
* 连接拷回来的光盘，然后利用ftp软件连接stout之后，上传数据。
* 所有的原始数据，都放到 /home/data/rawdata 文件夹下，每一个session用一个文件夹表示，通常为 ‘年月日_实验缩写_被试代号’的格式，例如“20220110_CET_CN003”
* 在改文件夹下，新建一个 `mat_files_from_exp`的文件夹用来存放所有的scan里面的行为数据

2. 打开terminal, 为是你自己上传，该文件夹只能你个人操作，开放权限让别人也能读取

In [72]:
import os,shutil

/home/data/rawdata/23BPhappy


In [ ]:
dirpath = '/home/data/rawdata/23BPhappy/' #获取数据存储文件夹，整理后的文件夹放在这
MRIpath = dirpath + 'MRI_data/' #获取原始MRI文件存储路径
Behavpath = dirpath + 'Behavior_data/' #获取原始行为数据存储路径
subjlist = os.listdir(MRIpath) #获取被试的初始编号，准备改成上述规定格式的编号
allmatlist = os.listdir(Behavpath) #获取全部被试的行为数据文件名，准备
print(subjlist)
%cd $dirpath

In [65]:
def copy_matfiles(this_subject,this_matpath,allmatlist): #从行为数据文件夹中找到这个被试的数据,并将其复制到/mat_files_from_exp文件夹
    matfilelist = []
    for i in allmatlist:
        if this_subject in i:
            matfilelist.append(i)

    if this_subject not in str(matfilelist):
        print('Matfiles of this_subject dont exist')
    else:
       if os.path.exists(this_matpath):
            for this_matfile in matfilelist:  
                print(this_matfile)
                shutil.copy(Behavpath + this_matfile, this_matpath + '/mat_files_from_exp/' + this_matfile)

In [69]:
%%capture
for subjfile in subjlist:
    this_subject = subjfile[0:4]
    #.开头的隐藏文件可能也会被识别，把它们排除
    if subjfile[0]==".": 
            continue
    #将文件夹命名为规定格式
    old_matpath = MRIpath + subjfile
    new_matpath = MRIpath + subjfile[-8:] + '_BPhappy_' + subjfile[0:4]
    os.rename(old_matpath,new_matpath) 
    #开放文件夹权限，并且使用dcm2niix_afni整理数据
    ! chgrp -R student $new_matpath  
    ! chmod g+w  $new_matpath
    ! dcm2niix_afni -f %t_%s_%p $new_matpath
    #准备/mat_files_from_exp文件夹
    if os.path.exists(new_matpath + '/mat_files_from_exp'):
        print('matdir exists!')
        continue
    os.mkdir(new_matpath + '/mat_files_from_exp') 
    #将行为数据复制到被试文件夹
    copy_matfiles(this_subject,new_matpath,allmatlist)

In [71]:
%cd $MRIpath
!ls -l
new_subjlist = os.listdir(MRIpath) #获取重命名后的被试文件夹列表 
for subjfolder in new_subjlist:
    if subjfolder[0]==".":
            continue
    print(subjfolder)
    pathname = MRIpath + subjfolder
    #进一步整理文件夹，在被试文件夹中，建立一个rawfiles文件夹，把分析不需要的图像移进去
    %cd $pathname
    ! mkdir rawfiles
    ! mv *localizer* rawfiles
    ! mv *_M*/ *_g*/ *_cmrr*/ -t rawfiles
    ! mv *DTI* rawfiles
    ! mv *TASK2* rawfiles
    ! mv *gre_field* rawfiles
    ! mv *REST* rawfiles

20230206_BPhappy_M002
/home/data/rawdata/23BPhappy/MRI_data/20230206_BPhappy_M002
20230227_BPhappy_M005
/home/data/rawdata/23BPhappy/MRI_data/20230227_BPhappy_M005
mv: cannot stat '*gre_field*': No such file or directory
20230227_BPhappy_M006
/home/data/rawdata/23BPhappy/MRI_data/20230227_BPhappy_M006
mv: cannot stat '*gre_field*': No such file or directory
20230220_BPhappy_M004
/home/data/rawdata/23BPhappy/MRI_data/20230220_BPhappy_M004
mv: cannot stat '*_M*/': No such file or directory
mv: cannot stat '*_g*/': No such file or directory
mv: cannot stat '*_cmrr*/': No such file or directory
mv: cannot stat '*DTI*': No such file or directory
mv: cannot stat '*TASK2*': No such file or directory
mv: cannot stat '*gre_field*': No such file or directory
20230313_BPhappy_M010
/home/data/rawdata/23BPhappy/MRI_data/20230313_BPhappy_M010
mv: cannot stat '*gre_field*': No such file or directory
20230312_BPhappy_M009
/home/data/rawdata/23BPhappy/MRI_data/20230312_BPhappy_M009
mv: cannot stat '*gr

In [73]:
%cd $MRIpath
! mv */ -t $dirpath #把整理的被试文件夹移动到指定目录

/home/data/rawdata/23BPhappy/MRI_data
